In [1]:
# Packages
import numpy as np
import matplotlib.pyplot as plt 

from scipy.optimize import minimize 
from math import log

In [2]:
# Parameters
# eq - empirical quantile
# mep - mean excess plot
init_method = 'eq'
init_level = 0.98
epsilon = 1e-8
num_candidate = 10

In [3]:
# Algorithm
def pot(data):
    # Parameters
    init_level = 0.98
    epsilon = 1e-8
    num_candidate = 10

    # Init threshold
    if init_method == 'eq':
        temp = np.sort(data)
        threshold = temp[int(init_level * data.size)]
    elif init_method == 'mep':
        threshold = mep(data)
    
    # Init Peaks
    peaks = data[data > threshold] - threshold

    # Fig
    # print(threshold)
    # fig, ax = plt.subplots(2)
    # ax[0].plot(data)
    # ax[1].plot(peaks)
    # plt.show()

    # Grimshaw
    min = peaks.min()
    max = peaks.max()
    mean = peaks.mean()

    print('min = {}\nmax = {}\nmean = {}'.format(min, max, mean))

    a = -1/max

    if abs(a) < 2 * epsilon:
        epsilon = abs(a)/num_candidate 
    
    a = a + epsilon
    b = 2 * (mean - min) / (mean * min)
    c = 2 * (mean - min) / (min ** 2)

    print('epsilon = {}'.format(epsilon))
    print('b = {}, c = {}'.format(b, c))

    left = solve(lambda t: function(peaks, threshold), lambda t: dev_function(peaks, threshold), (a + epsilon, -epsilon), num_candidate, 'regular')
    right = solve(lambda t: function(peaks, threshold), lambda t: dev_function(peaks, threshold), (b, c), num_candidate, 'regular')
    candidates = np.concatenate((left, right))

    gamma_best = 0
    sigma_best = 0
    ll_best = log_likelihood(peaks, gamma_best, sigma_best)

    for candidate in candidates:
        gamma = u(1 + z * peaks) - 1
        sigma = gamma / z
        ll = log_likelihood(peaks, gamma, sigma)
        if ll > ll_best:
            gamma_best = gamma
            sigma_best = sigma
            ll_best = ll

    # Cal threshold
    r = data.size * probability / peaks.size
    if gamma != 0:
        z = threshold + (sigma / gamma) * (pow(r, -gamma) - 1)
    else:
        z = threshold - sigma * log(r)

    return z 

def function(y, threshold):
    s = 1 + threshold * y
    us = 1 + np.log(s).mean()
    vs = np.mean(1/s)
    return us * vs - 1

def dev_function(y, threshold):
    s = 1 + threshold * y
    us = 1 + np.log(s).mean()
    vs = np.mean(1/s)
    dev_us = (1/threshold) * (1 - vs)
    dev_vs = (1/threshold) * (-vs + np.mean(1/s ** 2))
    return us * dev_vs + vs * dev_us

def solve(function, dev_function, bounds, num_points, method):
    if method == 'regular':
        step = (bounds[1]-bounds[0]) / (num_points + 1)

        print(bounds)
        print(step)

        x0 = np.arange(bounds[0] + step, bounds[1], step)
    elif method == 'ramdom':
        x0 = np.random.uniform(bounds[0], bounds[1], num_points)

    def obj_function(x, function, dev_function):
        g = 0
        j = np.zeros(x.shape)
        i = 0
        for item in x:
            fx = function(item)
            g = g + fx ** 2
            j[i] = 2 * fx * dev_function(item)
            i += i
        return g, j
    
    optimization = minimize(lambda x: obj_function(x, function, dev_function), x0, method='L-BFGS-B', jac=True, bounds=[bounds]*len(x0))
    x = optimization.x
    np.round(x, decimals=5) 
    return np.unique(x)

In [4]:
# Data
data = np.fromfile('../data/example1.dat')
print(data.shape)

(20386,)


In [5]:
# Test
output = pot(data)

min = 2.4733040147310453e+173
max = 1.6903554740103175e+178
mean = 3.8791959042489633e+177
epsilon = 5.915915411730114e-180
b = 0.0, c = 0.0
(-4.732732329384092e-179, -5.915915411730114e-180)
3.7646734438282554e-180
(0.0, 0.0)
0.0


ValueError: arange: cannot compute length